# Kaggle-Tutorial

## Geospatial Analysis

---

## 지리 공간 데이터 조작

장소의 이름만으로 위피를 찾는다. 그리고 공간 관계를 기반으로 데이터를 결합하는 방법을 뱅운다.

원본: [Manipulating Geospatial Data](https://www.kaggle.com/code/alexisbcook/manipulating-geospatial-data)
---
### 소개

이 튜토리얼에서는 지리 공간 데이터에 대한 두 가지 일밥적인 조작인 **geocoding** 과 **table joins**에 대해 배운다.

In [17]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')

### Geocoding

**Geocoding**은 장소의 이름이나 주소를 지도상의 위치로 변화하는 프로세스다. 예를 들어 [Google Maps](https://www.google.com/maps), [Bing Maps](https://www.bing.com/maps) 또는 [Baidu Maps](https://map.baidu.com/)를 사용하여 랜드마크 설명을 기반으로 지리적 위치를 조회한 적 있다면, 당신은 Geocoder를 사용한 것이다!

![](https://i.imgur.com/1IrgZQq.png)

우리는 geocoding을 위해 Geopy를 사용할 것이다.

In [18]:
from geopy.geocoders import Nominatim

위의 코드 셀에서 Nominatim은 위치를 생성하는데 사용할 geocoding 소프트웨어다.

먼저 geocoder를 인스턴스화한다. 그다음 이름이나 주소를 Python 문자열로 적용하기만 하면 된다. (이 경우 Giza의 대피라미드라고 알려진 "Khufu의 피라미드"를 가져왔다.)

Geocoing이 성공적이라면, 두 가지 중요한 속성attribute를 가진 `geopy.location.Location` 객체를 반환한다.

* "point" 속성attribute은 (위도latitude, 경도longitude) 위치를 포함하고,
* "address" 속성attribute은 전체 주소를 포함한다.

In [19]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

29 58m 44.9758s N, 31 8m 3.17634s E
هرم خوفو, شارع ابو الهول السياحي, نزلة البطران, الجيزة, 12556, مصر


"point" 속성의 값은 `geopy.point.Point` 객체이며, `latitude`와 `longitude` 속성에서 각각의 위도와 경도를 얻을 수 있다.

In [20]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 29.97915995
Longitude: 31.134215650388754


종종 많은 다른 주소들을 지오코딩해야하는 경우가 있다. 예를 들어 유럽의 상위 100갸 대학의 위치를 알고싶은 경우가 있다고 보자.

In [21]:
universities = pd.read_csv("./input/geospatial-learn-course-data/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


이때 lambda 함수를 이용하여 DataFrame의 모든 행에 지오코딩을 적용할 수 있다. (지오코딩에 실패한 경우를 설명하기 위해 try/except문을 사용한다.)

In [22]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

95.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,51.523581,-0.132977,POINT (-0.13298 51.52358)


다음으로 지오코더가 반환한 모든 위치를 시각화한다. 몇몇의 경우는 유럽에 있지 않기 때문에 위치가 정확하지 않을 수 있다.


In [23]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

### Table joins
이제 주제를 전환해서 서로 다른 소스의 데이터를 결합하는 방법에 대해 생각해보자.

#### 속성 결합Attribute join

여러 DataFrame의 정보를 공유 인덱스와 결합하기 위해 `pd.DataFrame.join()을 사용하는 방법은 이미 알고 있을 것이다. 우리는 이처럼 (인덱스에서 값들을 간단해 매칭시켜 주는 것으로) 테이터를 결합하는 방식을 ••attribute join••이라 한다.

GeoDataFrame과의 속성 결합을 할 때, 가장 좋은 방법은 `gpd.GeoDataFrame.merge()`를 사용하는 것이다. 이를 설명하기 위해 유럽의 모든 국가의 경계선을 포함한
 GeoDataFrame인 `europe_boundaries`로 작업합니다. 이 GeoDataFrame의 처음 5개 행은 아래에 표시되어 있다.

In [24]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [25]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


각 국가의 예상 인구 및 국내 총생산(GDP)이 포함된 DataFrame `europe_stats`와 결합해보자.

In [26]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


아래 코드 셀에서 속성 조인을 수행한다. `on` 인수argument는 `europe_boundaries`의 행을 `europe_stats`의 행과 일치시키는 데 사용되는 열 이름으로 설정된

In [27]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67106161,2699000.0
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",9960487,498100.0
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9549747,165400.0


공간 결합Spatial join
또 다른 유형의 결합은 **공간 결합Spatial join**이다. 공간 결합Spatial join을 통해, 우리는 "geometry" 열에 있는 객체들 간의 공간 관계공간 결합Spatial relationship를 기반으로GeoDataFrame을 결합한다. 예를 들어, 우리는 이미 유럽 대학의 지오코딩된 주소를 포함한 GeodataFrame인 `universities`를 가지고 있다.

이제 우리는 공간 결합Spatial join을 사용하여 각 대학을 해당 국가와 일치시킬 수 있다. 우리는 이를 `gpd.sjoin()`으로 해볼 것이다.

In [28]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

위의 공간 결합Spatial join은 두 GeoDataFrames의 "geomatry" 열을 살펴본다. `universities` GeoDataFrame의 Point 객체가 `europe` DataFrame의 Polygon 객체와 교차intersects하여 결합된 해당 행 `european_universities` DataFrame이 단일 행으로 추가된다. 서로 매칭되는 대학이 없는 국가(및 일치하는 국가가 없는 대학)의 경우 그 결과는 생략된다.

`gpd.sjoin()` 메서드는 `how` 및 `op` 인수argument를 통해 다양한 유형의 결합으로 사용자화 할 수 있다. 예를 들어, `how='left'`(또는 `how='right'`)를 설정하여 SQL left(또는 right) 결합과 같이 수행할 수 있다. 이 과정에서 자세한 내용은 다루지 않겠지만 더 자세히 알아보고자 한다면 [해당 문서](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)를 볼 수 있다.

### 당신의 차례

[**지오코딩과 테이블 결합**](https://www.kaggle.com/kernels/fork/5832170)을 사용하여 다음 스타벅스 리저브 로스터리에 적합한 위치를 식별하라.